# Data preparation

## Extract variables from raw experiment data

In [2]:
import sqlite3
import pandas as pd

from util import getExperimentSequence, getTaskCorrectness, getTaskDurationSeconds, getTaskFromLocation, getTaskSolution

cnx = sqlite3.connect('data/data_anon_after_data_integrity.db')

df = pd.read_sql_query("SELECT * FROM events", cnx)

df['task'] = df['location'].map(getTaskFromLocation)

results = pd.DataFrame(columns=[
    'student_id_anon',
    'seq',
    't0_time',
    't0_solution',
    't0_correctness',
    't1_time',
    't1_solution',
    't1_correctness',
    't2_time',
    't2_solution',
    't2_correctness',
    't3_time',
    't3_solution',
    't3_correctness',
    't4_time',
    't4_solution',
    't4_correctness',
    't5_time',
    't5_solution',
    't5_correctness',
    't6_time',
    't6_solution',
    't6_correctness',
    't7_time',
    't7_solution',
    't7_correctness',
    't8_time',
    't8_solution',
    't8_correctness',
    't9_time',
    't9_solution',
    't9_correctness',
])

for experimentId in df['experiment'].unique():
    tasks = list(range(10))

    seq = getExperimentSequence(experimentId, df)

    duration = list(map(lambda i: getTaskDurationSeconds(experimentId, i, df), tasks))
    solution = list(map(lambda i: getTaskSolution(experimentId, i, df), tasks))
    correctness = list(map(lambda i: getTaskCorrectness(experimentId, i, df), tasks))

    results = pd.concat([
        results,
        pd.DataFrame({
            'student_id_anon': experimentId,
            'seq': seq,
            't0_time': duration[0],
            't0_solution': solution[0],
            't0_correctness': correctness[0],
            't1_time': duration[1],
            't1_solution': solution[1],
            't1_correctness': correctness[1],
            't2_time': duration[2],
            't2_solution': solution[2],
            't2_correctness': correctness[2],
            't3_time': duration[3],
            't3_solution': solution[3],
            't3_correctness': correctness[3],
            't4_time': duration[4],
            't4_solution': solution[4],
            't4_correctness': correctness[4],
            't5_time': duration[5],
            't5_solution': solution[5],
            't5_correctness': correctness[5],
            't6_time': duration[6],
            't6_solution': solution[6],
            't6_correctness': correctness[6],
            't7_time': duration[7],
            't7_solution': solution[7],
            't7_correctness': correctness[7],
            't8_time': duration[8],
            't8_solution': solution[8],
            't8_correctness': correctness[8],
            't9_time': duration[9],
            't9_solution': solution[9],
            't9_correctness': correctness[9],
        }, index=[experimentId]
    )], ignore_index=True)

# One student did not finish in time and has null values for the last 5 tasks
results = results.dropna()

/var/folders/tq/ppy7x00d4kn0z3bgzscqy3mc0000gp/T/ipykernel_28744/4041879221.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([


## Calculate overall time and correctness

In [3]:
import statistics

def calculateTimeWrite(row, language):
    if language == 'jv':
        return statistics.mean([row['t1_time'], row['t3_time']]) if row['seq'] == 'AB' else statistics.mean([row['t2_time'], row['t4_time']])
    elif language == 'py':
        return statistics.mean([row['t1_time'], row['t3_time']]) if row['seq'] == 'BA' else statistics.mean([row['t2_time'], row['t4_time']])
    else:
        return None

def calculateTimeRead(row, language):
    if language == 'jv':
        return statistics.mean([row['t6_time'], row['t8_time']]) if row['seq'] == 'AB' else statistics.mean([row['t7_time'], row['t9_time']])
    elif language == 'py':
        return statistics.mean([row['t6_time'], row['t8_time']]) if row['seq'] == 'BA' else statistics.mean([row['t7_time'], row['t9_time']])
    else:
        return None

results['time_write_jv'] = results.apply(lambda row: calculateTimeWrite(row, 'jv'), axis=1)
results['time_write_py'] = results.apply(lambda row: calculateTimeWrite(row, 'py'), axis=1)
results['time_read_jv'] = results.apply(lambda row: calculateTimeRead(row, 'jv'), axis=1)
results['time_read_py'] = results.apply(lambda row: calculateTimeRead(row, 'py'), axis=1)

In [ ]:
import statistics

def calculateCorrectnessWrite(row, language):
    if language == 'jv':
        return statistics.mean([row['t1_correctness'], row['t3_correctness']]) if row['seq'] == 'AB' else statistics.mean([row['t2_correctness'], row['t4_correctness']])
    elif language == 'py':
        return statistics.mean([row['t1_correctness'], row['t3_correctness']]) if row['seq'] == 'BA' else statistics.mean([row['t2_correctness'], row['t4_correctness']])
    else:
        return None

def calculateCorrectnessRead(row, language):
    if language == 'jv':
        return statistics.mean([row['t6_correctness'], row['t8_correctness']]) if row['seq'] == 'AB' else statistics.mean([row['t7_correctness'], row['t9_correctness']])
    elif language == 'py':
        return statistics.mean([row['t6_correctness'], row['t8_correctness']]) if row['seq'] == 'BA' else statistics.mean([row['t7_correctness'], row['t9_correctness']])
    else:
        return None

results['correctness_write_jv'] = results.apply(lambda row: calculateCorrectnessWrite(row, 'jv'), axis=1)
results['correctness_write_py'] = results.apply(lambda row: calculateCorrectnessWrite(row, 'py'), axis=1)
results['correctness_read_jv'] = results.apply(lambda row: calculateCorrectnessRead(row, 'jv'), axis=1)
results['correctness_read_py'] = results.apply(lambda row: calculateCorrectnessRead(row, 'py'), axis=1)

In [5]:
results.to_csv('./data/results_anon.csv', index=False)

results

,student_id_anon,seq,t0_time,t0_solution,t0_correctness,t1_time,t1_solution,t1_correctness,t2_time,t2_solution,...,t9_solution,t9_correctness,time_write_jv,time_write_py,time_read_jv,time_read_py,correctness_write_jv,correctness_write_py,correctness_read_jv,correctness_read_py
0,48,AB,93.5160,block ExampleDataSelector oftype CellRangeSel...,0,60.1250,A2:K4;,0,226.028,"0:3,0:4",...,"[[4, 5], [5, 5], [6, 5], [7, 5], [8, 5], [8, 6...",0.800000,52.24050,130.6920,69.6140,56.5655,0.5,0.633333,0.785714,0.816667
1,96,BA,75.4560,"delete rows A, B, C, G, H, I, J. delete column...",0,93.3500,"1:4, :",1.0,81.558,A1:D5,...,"[[7, 8], [5, 4], [4, 4], [4, 6], [4, 7], [5, 7...",0.888889,77.09800,67.9105,28.9100,23.5630,1.0,1.000000,0.898990,0.928571
2,12,AB,86.1010,D4:F6,0.03,56.9190,A2:*4,1.0,261.467,"[0,1, 2,3, 4] , [0,1, 2,3]",...,"[[4, 4], [4, 5], [4, 6], [4, 7], [4, 8], [5, 4...",0.800000,49.36150,174.8915,64.7930,36.1535,1.0,0.500000,0.928571,0.854545
3,59,AB,165.2858,block ExampleDataSelector oftype CellRangeSele...,0,148.7004,A2:J4,1.0,108.628,":3,:4",...,"[[4, 4], [4, 5], [4, 6], [4, 7], [4, 8], [5, 4...",0.888889,159.67715,138.8401,36.1557,40.2018,1.0,0.800000,0.928571,0.898990
4,95,AB,91.7680,select range: C4:E6;,0,133.2350,B1:J4,0,342.598,:3,...,"[[4, 4], [5, 4], [6, 4], [4, 6], [4, 7], [5, 7...",0.800000,80.85300,240.0095,63.2705,106.3510,0.5,0.157895,0.928571,0.757143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,35,AB,15.5780,xyz,0,153.8520,A2:J4,1.0,242.604,"0:5,5",...,"[[5, 4], [6, 4], [7, 4], [8, 4], [5, 5], [6, 5...",0.800000,99.91050,186.3920,78.1880,63.9245,1.0,0.000000,0.928571,0.816667
89,85,AB,356.2870,block ExampleDataSelector oftype CellRangeSel...,0,370.0610,block CarsCoreDataSelector oftype CellRangeSe...,0,139.830,"[A1*A3],[E1*E3]",...,"[[5, 3], [3, 4], [5, 4], [5, 5], [3, 5], [4, 6...",0.700000,299.16000,102.4580,11.6010,8.5295,0.0,0.000000,0.607143,0.528571
90,38,AB,214.6150,block ExampleDataSelector oftype CellRangeSel...,0,240.8140,block ExampleRowDeleter oftype RowDeleter { ...,0,375.554,df1.iloc[4:5],...,"[[7, 3], [9, 3]]",0.363636,199.55650,257.9475,69.4245,46.6925,0.0,0.076923,0.714286,0.431818
91,88,BA,241.6160,select: range D4:F6,0.03,418.1400,"2,10",0,56.788,A1:D5,...,"[[4, 4], [7, 8], [4, 5], [4, 6], [4, 7], [4, 8...",0.888889,61.51650,239.7950,50.6410,75.1665,1.0,0.000000,0.853535,0.571429
